# MMM 18-02-2025 Tag-up meeting demo Notebook

This notebook demonstrates various functionalities using MMS (Magnetospheric Multiscale) mission data analysis with SciQLop. It includes:

- Basic data visualization of MMS1 magnetic field, ion density and energy spectrogram
- Signal processing with custom virtual products for filtering high-frequency electric field data 
- Time-shift operations on spacecraft data
- Visualization of spacecraft trajectories with magnetopause and bow shock models

The examples use the SciQLop Python API along with the speasy library for data access and processing.


In [ ]:
from typing import List, Optional
from SciQLop.user_api.plot import *
from SciQLop.user_api.virtual_products import create_virtual_product, VirtualProductType
from datetime import datetime, timedelta
import speasy as spz
from speasy import SpeasyVariable
from speasy.signal.filtering import sosfiltfilt
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
%matplotlib inline

# Simple plot panel

In this panel, we create a basic visualization of MMS1 data showing:
- Magnetic field components from FGM instrument
- Ion density from FPI instrument 
- Ion energy spectrogram from FPI instrument

The time range is set to observe data during December 30-31, 2018.


In [ ]:
p = create_plot_panel()
p.plot("speasy//cda//MMS//MMS1//FGM//MMS1_FGM_SRVY_L2//mms1_fgm_b_gse_srvy_l2")
p.plot("speasy//cda//MMS//MMS1//DIS//MMS1_FPI_FAST_L2_DIS_MOMS//mms1_dis_numberdensity_fast")
p.plot("speasy//cda//MMS//MMS1//DIS//MMS1_FPI_FAST_L2_DIS_MOMS//mms1_dis_energyspectr_omni_fast")
p.time_range = TimeRange("2018-12-30", "2018-12-31")

# Virtual Product

In this section, we create a custom virtual product to filter high-frequency electric field data from MMS1. The implementation includes:
- IIR bandpass filter design and visualization 
- Virtual product creation for filtered E-field
- Comparison plots between raw and filtered signals


In [ ]:
sos = signal.iirfilter(N=8, Wn=[100 / 32000, 10000 / 32000], rs=80, btype='bandpass', output='sos')
w, h = signal.sosfreqz(sos, worN=150000)
plt.subplot(2, 1, 1)
db = 20 * np.log10(np.maximum(np.abs(h), 1e-5))
plt.plot(w / np.pi, db)
plt.ylim(-75, 5)
plt.grid(True)
plt.yticks([0, -20, -40, -60])
plt.semilogx()
plt.ylabel('Gain [dB]')
plt.title('Frequency Response')
plt.subplot(2, 1, 2)
plt.plot(w / np.pi, np.angle(h))
plt.grid(True)
plt.yticks([-np.pi, -0.5 * np.pi, 0, 0.5 * np.pi, np.pi],
           [r'$-\pi$', r'$-\pi/2$', '0', r'$\pi/2$', r'$\pi$'])
plt.ylabel('Phase [rad]')
plt.xlabel('Normalized frequency (1.0 = Nyquist)')
plt.semilogx()
plt.show()



In [ ]:
def filter_mms1_edp_hmfe_par_epar_brst_l2(start: datetime, stop: datetime) -> Optional[SpeasyVariable]:
    B = spz.get_data(spz.inventories.tree.cda.MMS.MMS1.ADP_SDP.MMS1_EDP_BRST_L2_HMFE.mms1_edp_hmfe_par_epar_brst_l2,
                     start - timedelta(seconds=1),
                     stop + timedelta(seconds=1))
    b = sosfiltfilt(sos, B)
    return b[start:stop]


filtered_mms1_edp_hmfe_par_epar_brst_l2 = create_virtual_product(path="MMS1/filter_mms1_edp_hmfe_par_epar_brst_l2",
                                                                 callback=filter_mms1_edp_hmfe_par_epar_brst_l2,
                                                                 product_type=VirtualProductType.Scalar,
                                                                 labels=["filtered DC E parallel"],
                                                                 cachable=True)

p = create_plot_panel()
p.time_range = TimeRange("2019-02-17T12:33", "2019-02-17T12:34")
p.plot("speasy//cda//MMS//MMS1//ADP_SDP//MMS1_EDP_BRST_L2_HMFE//mms1_edp_hmfe_par_epar_brst_l2")
p.plot(filtered_mms1_edp_hmfe_par_epar_brst_l2)

# Simple Constant Time Shift Example (2015-12-28 22h)

This section demonstrates how to create a time-shifted version of MMS3 FGM data. The example:
- Shifts MMS3 magnetic field data by 1 second
- Creates a virtual product for the shifted data
- Plots the original and shifted data for comparison


In [ ]:
def shifted_mms3_fgm(start: datetime, stop: datetime) -> Optional[SpeasyVariable]:
    mms2_fgm = spz.get_data(spz.inventories.tree.cda.MMS.MMS3.FGM.MMS3_FGM_SRVY_L2.mms3_fgm_b_gse_srvy_l2, start, stop)
    if mms2_fgm is None:
        return None
    return (mms2_fgm + np.timedelta64(1, 's'))["Bt"]


shifted_mms3_fgm = create_virtual_product(path="MMS1/shifted_mms3_fgm",
                                          callback=shifted_mms3_fgm,
                                          product_type=VirtualProductType.Scalar,
                                          labels=["|b| - 1s"],
                                          cachable=True)

p = create_plot_panel()
p.time_range = TimeRange("2015-12-28T22:11", "2015-12-28T22:13")
plot, graph = p.plot("speasy//amda//Parameters//MMS//MMS3//FGM//survey//|b|")
plot.plot(shifted_mms3_fgm)

# Trajectories + Models Visualization

This section demonstrates how to visualize MMS spacecraft trajectories along with magnetopause and bow shock models. The following cells will:

1. Create a virtual product for MMS1 position data in GSE coordinates
2. Define callbacks for magnetopause and bow shock models using [spok](https://github.com/LaboratoryOfPlasmaPhysics/spok)
3. Plot the spacecraft trajectory with model boundaries in 2D projections


In [ ]:
def mms1_pos_gse(start: float, stop: float):
    try:
        v = spz.get_data(f"amda/mms1_xyz_gse", start, stop)
        if v is None:
            return None
        t = (v.time.astype(np.int64) / 1e9).astype(np.float64)
        x = v.values[:, 0].astype(np.float64)
        y = v.values[:, 1].astype(np.float64)
        z = v.values[:, 2].astype(np.float64)
        return t, x, y, z
    except Exception as e:
        print(f"Error: {e}")
        return None


vp = create_virtual_product(path="MMS1/pos",
                            callback=mms1_pos_gse,
                            product_type=VirtualProductType.MultiComponent,
                            labels=["MMS1 GSE"] * 3,
                            cachable=True)

In [ ]:
from spok.models.planetary import Magnetosheath

msh = Magnetosheath(magnetopause='mp_shue1998', bow_shock='bs_jelinek2012')
index = np.linspace(-0.8 * np.pi, 0.8 * np.pi, 100)


def magnetopause_cb(start: float, stop: float) -> List[np.ndarray]:
    Pd = np.nanmean(spz.get_data(
        spz.inventories.tree.amda.Parameters.OMNI.Sun__Solar_Wind__Ground_Based_Indices.omni_1min_v2.omni_hro2_1min_sw_p,
        start, stop))
    Bz = np.nanmean(spz.get_data(
        spz.inventories.tree.amda.Parameters.OMNI.Sun__Solar_Wind__Ground_Based_Indices.omni_1min_v2.omni_hro2_1min_b_gse,
        start, stop)['bz'])
    A = msh.magnetopause(index, np.pi / 2, Pd=Pd, Bz=Bz)
    B = msh.magnetopause(np.pi / 2, index, Pd=Pd, Bz=Bz)
    C = msh.magnetopause(index, 0, Pd=Pd, Bz=Bz)
    return A[0].copy(), A[1].copy(), B[1].copy(), B[2].copy(), C[0].copy(), C[2].copy()


def bow_shock_cb(start: float, stop: float) -> List[np.ndarray]:
    Pd = np.nanmean(spz.get_data(
        spz.inventories.tree.amda.Parameters.OMNI.Sun__Solar_Wind__Ground_Based_Indices.omni_1min_v2.omni_hro2_1min_sw_p,
        start, stop))
    Bz = np.nanmean(spz.get_data(
        spz.inventories.tree.amda.Parameters.OMNI.Sun__Solar_Wind__Ground_Based_Indices.omni_1min_v2.omni_hro2_1min_b_gse,
        start, stop)['bz'])
    A = msh.bow_shock(index, np.pi / 2, Pd=Pd, Bz=Bz)
    B = msh.bow_shock(np.pi / 2, index, Pd=Pd, Bz=Bz)
    C = msh.bow_shock(index, 0, Pd=Pd, Bz=Bz)
    return A[0].copy(), A[1].copy(), B[1].copy(), B[2].copy(), C[0].copy(), C[2].copy()


vp = create_virtual_product(path="models/magnetopause",
                            callback=magnetopause_cb,
                            product_type=VirtualProductType.MultiComponent,
                            labels=["ms x,y", "ms y,z", "ms x,z"],
                            cachable=True)

vp = create_virtual_product(path="models/bow shock",
                            callback=bow_shock_cb,
                            product_type=VirtualProductType.MultiComponent,
                            labels=["bs x,y", "bs y,z", "bs x,z"],
                            cachable=True)


In [ ]:
panel = create_plot_panel()
plot, graph = panel.plot("MMS1/pos", plot_type=PlotType.Projection)
plot.plot("models/magnetopause")
plot.plot("models/bow shock")
panel.plot("speasy//cda//MMS//MMS1//FGM//MMS1_FGM_SRVY_L2//mms1_fgm_b_gse_srvy_l2")
panel.time_range = TimeRange("2015-10-10", "2015-10-11")